In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from imutils import paths
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os

In [2]:
datasetPath='..\kvasir-dataset'
categories=[
    'dyed-lifted-polyps', 
    'dyed-resection-margins',
    'esophagitis',
    'normal-cecum',
    'normal-pylorus',
    'normal-z-line',
    'polyps',
    'ulcerative-colitis'
]
EPOCHS = 200
Learing_Rate = 1e-3
Batch_Size = 128

In [3]:
def buildModel(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    if backend.image_data_format() == "channels_first":
        inputShape = (depth, height, width)
    # CONV => RELU => POOL layers
    model.add(Conv2D(16, (5, 5), padding="same",input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU => POOL layers
    model.add(Conv2D(32, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # FC => RELU layers
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    # softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax"))
    return model

In [4]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    if iteration == total: 
        print()

In [5]:
data = []
labels = []

print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(datasetPath)))
random.seed(42)
random.shuffle(imagePaths)

printProgressBar(0, len(imagePaths), prefix = 'Progress:', suffix = 'Complete', length = 50)
for i,imagePath in enumerate(imagePaths):
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (96, 96))
    image = img_to_array(image)
    data.append(image)
    label = imagePath.split(os.path.sep)[-2]
    label = categories.index(label)
    labels.append(label)
    printProgressBar(i + 1, len(imagePaths), prefix = 'Progress:', suffix = 'Complete', length = 50)

print("[INFO] loading images completed...")

[INFO] loading images...
Progress: |██████████████████████████████████████████████████| 100.0% Complete
[INFO] loading images completed...


In [6]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)

trainY = to_categorical(trainY, num_classes=8)
testY = to_categorical(testY, num_classes=8)

In [7]:
print ("number of training examples = " + str(trainX.shape[0]))
print ("number of test examples = " + str(testX.shape[0]))
print ("X_train shape: " + str(trainX.shape))
print ("Y_train shape: " + str(trainY.shape))
print ("X_test shape: " + str(testX.shape))
print ("Y_test shape: " + str(testY.shape))

number of training examples = 3000
number of test examples = 1000
X_train shape: (3000, 96, 96, 3)
Y_train shape: (3000, 8)
X_test shape: (1000, 96, 96, 3)
Y_test shape: (1000, 8)


In [8]:
generator = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

In [9]:
print("[INFO] compiling model...")
strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
model = buildModel(width=96, height=96, depth=3, classes=8)
model.summary()

[INFO] compiling model...
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 16)        1216      
_________________________________________________________________
activation (Activation)      (None, 96, 96, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        12832     
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0    

In [10]:
opt = Adam(lr=Learing_Rate, decay=Learing_Rate / EPOCHS)
model.compile(
    loss="binary_crossentropy", 
    optimizer=opt, 
    metrics=[
        "accuracy",
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.SpecificityAtSensitivity(0.5,name="specificity")
    ]
)

In [11]:
history = model.fit(
    x=generator.flow(trainX, trainY, batch_size=Batch_Size),
    validation_data=(testX, testY), 
    steps_per_epoch=len(trainX) // Batch_Size,
    epochs=EPOCHS, 
    verbose=1)
print("[INFO] training network completed...")

Epoch 1/200
23/23 [==============================] - 25s 1s/step - loss: 0.2998 - accuracy: 0.3962 - precision: 0.5579 - recall: 0.1560 - specificity: 0.8947 - val_loss: 0.2251 - val_accuracy: 0.5490 - val_precision: 0.5908 - val_recall: 0.4750 - val_specificity: 0.9447
Epoch 2/200
23/23 [==============================] - 28s 1s/step - loss: 0.2204 - accuracy: 0.5439 - precision: 0.6162 - recall: 0.3628 - specificity: 0.9485 - val_loss: 0.1985 - val_accuracy: 0.6060 - val_precision: 0.6344 - val_recall: 0.4720 - val_specificity: 0.9561
Epoch 3/200
23/23 [==============================] - 26s 1s/step - loss: 0.1892 - accuracy: 0.6076 - precision: 0.6571 - recall: 0.5084 - specificity: 0.9627 - val_loss: 0.1891 - val_accuracy: 0.6140 - val_precision: 0.6358 - val_recall: 0.5360 - val_specificity: 0.9623
Epoch 4/200
23/23 [==============================] - 26s 1s/step - loss: 0.1813 - accuracy: 0.6299 - precision: 0.6773 - recall: 0.5446 - specificity: 0.9674 - val_loss: 0.1784 - val_accu

23/23 [==============================] - 28s 1s/step - loss: 0.1244 - accuracy: 0.7643 - precision: 0.7807 - recall: 0.7448 - specificity: 0.9906 - val_loss: 0.1464 - val_accuracy: 0.7310 - val_precision: 0.7402 - val_recall: 0.7150 - val_specificity: 0.9853
Epoch 32/200
23/23 [==============================] - 28s 1s/step - loss: 0.1241 - accuracy: 0.7643 - precision: 0.7766 - recall: 0.7458 - specificity: 0.9906 - val_loss: 0.1404 - val_accuracy: 0.7400 - val_precision: 0.7518 - val_recall: 0.7270 - val_specificity: 0.9870
Epoch 33/200
23/23 [==============================] - 29s 1s/step - loss: 0.1293 - accuracy: 0.7542 - precision: 0.7678 - recall: 0.7333 - specificity: 0.9900 - val_loss: 0.1556 - val_accuracy: 0.7180 - val_precision: 0.7319 - val_recall: 0.6960 - val_specificity: 0.9844
Epoch 34/200
23/23 [==============================] - 29s 1s/step - loss: 0.1233 - accuracy: 0.7705 - precision: 0.7847 - recall: 0.7500 - specificity: 0.9912 - val_loss: 0.1450 - val_accuracy: 0.7

23/23 [==============================] - 28s 1s/step - loss: 0.1029 - accuracy: 0.8189 - precision: 0.8272 - recall: 0.8068 - specificity: 0.9954 - val_loss: 0.1394 - val_accuracy: 0.7610 - val_precision: 0.7652 - val_recall: 0.7530 - val_specificity: 0.9906
Epoch 62/200
23/23 [==============================] - 27s 1s/step - loss: 0.0969 - accuracy: 0.8281 - precision: 0.8365 - recall: 0.8169 - specificity: 0.9962 - val_loss: 0.1505 - val_accuracy: 0.7520 - val_precision: 0.7589 - val_recall: 0.7430 - val_specificity: 0.9880
Epoch 63/200
23/23 [==============================] - 27s 1s/step - loss: 0.0980 - accuracy: 0.8269 - precision: 0.8335 - recall: 0.8172 - specificity: 0.9963 - val_loss: 0.1582 - val_accuracy: 0.7320 - val_precision: 0.7387 - val_recall: 0.7210 - val_specificity: 0.9866
Epoch 64/200
23/23 [==============================] - 29s 1s/step - loss: 0.1018 - accuracy: 0.8162 - precision: 0.8229 - recall: 0.8043 - specificity: 0.9958 - val_loss: 0.1478 - val_accuracy: 0.7

23/23 [==============================] - 25s 1s/step - loss: 0.0808 - accuracy: 0.8618 - precision: 0.8689 - recall: 0.8562 - specificity: 0.9982 - val_loss: 0.1489 - val_accuracy: 0.7630 - val_precision: 0.7671 - val_recall: 0.7610 - val_specificity: 0.9903
Epoch 92/200
23/23 [==============================] - 25s 1s/step - loss: 0.0796 - accuracy: 0.8519 - precision: 0.8582 - recall: 0.8431 - specificity: 0.9987 - val_loss: 0.1751 - val_accuracy: 0.7170 - val_precision: 0.7202 - val_recall: 0.7130 - val_specificity: 0.9860
Epoch 93/200
23/23 [==============================] - 25s 1s/step - loss: 0.0890 - accuracy: 0.8454 - precision: 0.8530 - recall: 0.8402 - specificity: 0.9971 - val_loss: 0.1452 - val_accuracy: 0.7540 - val_precision: 0.7584 - val_recall: 0.7440 - val_specificity: 0.9873
Epoch 94/200
23/23 [==============================] - 25s 1s/step - loss: 0.0839 - accuracy: 0.8506 - precision: 0.8568 - recall: 0.8416 - specificity: 0.9982 - val_loss: 0.1494 - val_accuracy: 0.7

23/23 [==============================] - 25s 1s/step - loss: 0.0685 - accuracy: 0.8802 - precision: 0.8867 - recall: 0.8743 - specificity: 0.9994 - val_loss: 0.1523 - val_accuracy: 0.7720 - val_precision: 0.7780 - val_recall: 0.7710 - val_specificity: 0.9920
Epoch 122/200
23/23 [==============================] - 25s 1s/step - loss: 0.0672 - accuracy: 0.8900 - precision: 0.8957 - recall: 0.8851 - specificity: 0.9992 - val_loss: 0.1692 - val_accuracy: 0.7450 - val_precision: 0.7487 - val_recall: 0.7360 - val_specificity: 0.9899
Epoch 123/200
23/23 [==============================] - 25s 1s/step - loss: 0.0674 - accuracy: 0.8809 - precision: 0.8858 - recall: 0.8753 - specificity: 0.9992 - val_loss: 0.1547 - val_accuracy: 0.7720 - val_precision: 0.7755 - val_recall: 0.7670 - val_specificity: 0.9896
Epoch 124/200
23/23 [==============================] - 25s 1s/step - loss: 0.0654 - accuracy: 0.8858 - precision: 0.8894 - recall: 0.8820 - specificity: 0.9993 - val_loss: 0.1602 - val_accuracy: 

23/23 [==============================] - 26s 1s/step - loss: 0.0574 - accuracy: 0.9060 - precision: 0.9079 - recall: 0.9032 - specificity: 0.9994 - val_loss: 0.1874 - val_accuracy: 0.7570 - val_precision: 0.7616 - val_recall: 0.7540 - val_specificity: 0.9879
Epoch 152/200
23/23 [==============================] - 26s 1s/step - loss: 0.0692 - accuracy: 0.8868 - precision: 0.8893 - recall: 0.8837 - specificity: 0.9989 - val_loss: 0.1656 - val_accuracy: 0.7750 - val_precision: 0.7795 - val_recall: 0.7740 - val_specificity: 0.9900
Epoch 153/200
23/23 [==============================] - 26s 1s/step - loss: 0.0531 - accuracy: 0.9119 - precision: 0.9154 - recall: 0.9081 - specificity: 0.9999 - val_loss: 0.1676 - val_accuracy: 0.7790 - val_precision: 0.7805 - val_recall: 0.7750 - val_specificity: 0.9910
Epoch 154/200
23/23 [==============================] - 25s 1s/step - loss: 0.0529 - accuracy: 0.9133 - precision: 0.9174 - recall: 0.9088 - specificity: 0.9996 - val_loss: 0.1708 - val_accuracy: 

KeyboardInterrupt: 

In [13]:
print("[INFO] saving model...")
model.save('Model_CNN3-1.model', save_format="h5")
print("[INFO] model saved")

[INFO] saving model...
[INFO] model saved


In [14]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch No")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()
plt.savefig('plot_CNN3-1.png')

NameError: name 'history' is not defined

<Figure size 432x288 with 0 Axes>

In [12]:
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["precision"], label="train_precision")
plt.plot(np.arange(0, N), history.history["recall"], label="train_recall")
plt.plot(np.arange(0, N), history.history["specificity"], label="train_specificity")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), history.history["val_precision"], label="val_precision")
plt.plot(np.arange(0, N), history.history["val_recall"], label="val_recall")
plt.plot(np.arange(0, N), history.history["val_specificity"], label="val_specificity")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_accuracy")
plt.title("Training Metrics")
plt.xlabel("Epoch No")
plt.ylabel("Value")
plt.legend(loc="lower left")
plt.show()
plt.savefig('plot_CNN3_metrics-1.png')

NameError: name 'history' is not defined

<Figure size 432x288 with 0 Axes>